In [216]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import KMeans,AgglomerativeClustering,Birch
from sklearn.mixture import GaussianMixture
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.manifold import TSNE
from sklearn.random_projection import GaussianRandomProjection
from sklearn.preprocessing import StandardScaler

# Wczytanie danych

In [217]:

labels = pd.read_fwf("./UCI_HAR_Dataset/features.txt",header=None)
X_train = pd.read_fwf("./UCI_HAR_Dataset/train/X_train.txt",header=None)
X_train.columns = labels


y_train = pd.read_fwf("./UCI_HAR_Dataset/train/y_train.txt",header=None)
y_train = pd.Series(y_train[0])

X_test = pd.read_fwf("./UCI_HAR_Dataset/test/X_test.txt",header=None)
X_test.columns = labels

y_test = pd.read_fwf("./UCI_HAR_Dataset/test/y_test.txt",header=None)
y_test = pd.Series(y_test[0])

data = pd.concat([X_train,X_test])
y = pd.concat([y_train,y_test])


In [218]:
proper = list()
for i in range(len(data.columns)):
    proper.append(data.columns[i][0])  
data.columns = proper
k = 6
data.reset_index(inplace=True,drop=True)

## Baseline

Jako punkt odniesniesienia do wszelkich następnych prób sprawdźmy jak poradzą sobie podstawowe algorytmy na nieprzekstałconym zbiorze danych.

In [219]:
kmean = KMeans(k)
yhat = kmean.fit_predict(data)
adjusted_rand_score(y,yhat)

0.46137426832867395

In [220]:
ac = AgglomerativeClustering(k)
yhat = ac.fit_predict(data) 
adjusted_rand_score(y,yhat)

0.5637552806358106

In [221]:
bir = Birch(n_clusters = k)
yhat = bir.fit_predict(data) 
adjusted_rand_score(y,yhat)

0.5637552806358106

Jak widać wyniki osiągane przez algorytmy nie są najlepsze, czemu z pewnością nie sprzyaja szum tworzony przez sporą ilość kolumn

## Feature engineering

Biorąc pod uwagę z jak dużą ilościa kolumn mamy do czynienia nie ma mowy o ręcznym wybieraniu cech, musimy się posiłkować odpowiednimi algorytmami.

In [222]:
pca = PCA(n_components=100)
pca_data = pca.fit_transform(data)
cum_ratio = (np.cumsum(pca.explained_variance_ratio_))

variance_ratio = 0.80
variance = np.cumsum(pca.explained_variance_ratio_)
num = np.sum(variance<variance_ratio)+1
print(f'{variance_ratio} % wariancji jest przestawiane przez {num} zmiennych')

0.8 % wariancji jest przestawiane przez 11 zmiennych


In [223]:
pca = PCA(n_components=11)
pca_data = pca.fit_transform(data)
kmean = KMeans(k)
yhat = kmean.fit_predict(pca_data)
adjusted_rand_score(y,yhat)

0.46246309508542954

## SelectKBest

Kolejną metoda z której zdecydujemy się skorzystać będzie SelectKBest z pakietu sklearn

In [224]:
X_new = SelectKBest(k=11).fit_transform(data,y)

In [225]:
kmean = KMeans(k)
yhat = kmean.fit_predict(X_new)
adjusted_rand_score(y,yhat)

0.5056726107998837

## PFA

In [226]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from collections import defaultdict
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler

class PFA(object):
    def __init__(self, n_features, q=None):
        self.q = q
        self.n_features = n_features

    def fit(self, X):
        if not self.q:
            self.q = X.shape[1]

        X = np.array(X)
       

        pca = PCA(n_components=self.q).fit(X)
        A_q = pca.components_.T

        kmeans = KMeans(n_clusters=self.n_features).fit(A_q)
        clusters = kmeans.predict(A_q)
        cluster_centers = kmeans.cluster_centers_

        dists = defaultdict(list)
        for i, c in enumerate(clusters):
            dist = euclidean_distances([A_q[i, :]], [cluster_centers[c, :]])[0][0]
            dists[c].append((i, dist))

        self.indices_ = [sorted(f, key=lambda x: x[1])[0][0] for f in dists.values()]
        self.features_ = X[:, self.indices_]


In [227]:
pfa = PFA(n_features=11)
pfa.fit(data)

data_prob = pfa.features_

kmean = KMeans(k)
yhat = kmean.fit_predict(X)
adjusted_rand_score(y,yhat)

0.39245733179618286